### Generating names with recurrent neural networks (5 points)

This time you'll find yourself delving into the heart (and other intestines) of recurrent neural networks on a class of toy problems.

Struggle to find a name for the variable? Let's see how you'll come up with a name for your son/daughter. Surely no human has expertize over what is a good child name, so let us train RNN instead;

It's dangerous to go alone, take these:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
start_token = " "

with open("news.txt",encoding='cp1251') as f:
    names = f.read()[:-1].split('\n')
    names = [start_token+name for name in names]


#print ('n samples = ',len(names))
for x in names[::1000]:
    print (x)

MAX_LENGTH = max(map(len,names))
#print("max length =", MAX_LENGTH)

#plt.title('Sequence length distribution')
#plt.hist(list(map(len,names)),bins=25);

#all unique characters go here
#tokens = sorted(set(' '.join(names)))

tokens=[" "]
token=""
f=open("news.txt",encoding='cp1251')
for line in list(f):
    for i in line:
        if(i==' ' or i=='\n'):
            tokens.append(token)
            token=''
        else:
            token+=i
"""f = open('Words_news.txt', 'w',encoding='cp1251')
for i in tokens:
    f.write(i)
    f.write("\n")
f.close()"""

#tokens = sorted(set(' '.join(names)))

ts=(" ".join(tokens)).split()
ts.append(" ")
tokens = sorted(set(list(ts)))

n_tokens = len(tokens)
#print ('n_tokens = ',n_tokens)

#assert 50 < n_tokens < 60

token_to_id = {token:ix for ix,token in enumerate(tokens)}

assert len(tokens) == len(token_to_id), "dictionaries must have same size"

for i in range(n_tokens):
    assert token_to_id[tokens[i]] == i, "token identifier must be it's position in tokens list"

print("Seems alright!")

def to_matrix(names,max_len=None,pad=token_to_id[' '],dtype='int32'):
    """Casts a list of names into rnn-digestable matrix"""
    
    max_len = max_len or max(map(len,names))
    names_ix = np.zeros([len(names),max_len],dtype) + pad
    #print(np.zeros([len(names),max_len],dtype) + pad)
    
    for i in range(len(names)):
        name_ix = list(map(token_to_id.get,names[i].split()))
        #print(list(map(token_to_id.get,names[i].split())))
        names_ix[i,:len(name_ix)] = name_ix
        #print(names_ix[i,:len(name_ix)])


    return names_ix.T

#Example: cast 4 random names to matrices, pad with zeros
#print('\n'.join(names[::2000]))
#print(to_matrix(names[::2000]).T)

!pip3 install https://github.com/lasagne/lasagne/archive/master.zip
!pip3 install https://github.com/yandexdataschool/agentnet/archive/master.zip

import theano
theano.config.floatX = 'float32'
import theano.tensor as T
import lasagne
import lasagne.layers as L

n_tokens = len(tokens)
rnn_num_units = 256
embedding_size = 64

from agentnet.memory import LSTMCell

def log_softmax(a, axis=-1):
    return a - T.log(T.exp(a).sum(axis=axis, keepdims=True))

prev_token = L.InputLayer([None])
prev_rnn = L.InputLayer([None, rnn_num_units])
prev_rnn1 = L.InputLayer([None, rnn_num_units])

# convert character id into embedding

prev_token_emb = L.EmbeddingLayer(prev_token, n_tokens, embedding_size)

# concatenate x embedding and previous h state
#rnn_input = L.ConcatLayer([prev_token_emb, prev_rnn])

# compute next state given x_and_h

#new_rnn = L.DenseLayer(rnn_input, rnn_num_units, nonlinearity=T.tanh)


(new_rnn,new_rnn1) = LSTMCell(prev_rnn,prev_rnn1,prev_token_emb) #GRUCell(prev_rnn1,[new_rnn])

# get probabilities for language model P(x_next|h_next)
next_token_logits = L.DenseLayer(new_rnn1, n_tokens, nonlinearity=None) #L.ConcatLayer([new_rnn,new_rnn1])

next_token_probs = L.NonlinearityLayer(next_token_logits, T.nnet.softmax)
next_token_logprobs = L.NonlinearityLayer(next_token_logits, log_softmax)

input_sequence = T.imatrix("input tokens [time, batch]")
batch_size = input_sequence.shape[1]

predicted_probas = []
h0 = T.zeros([batch_size,rnn_num_units]) #initial hidden state
h1 = T.zeros([batch_size,rnn_num_units])
probas0 = T.zeros([batch_size, n_tokens])

state0 = [h0,h1, probas0]

def rnn_one_step(x_t, h_t, h1_t, prev_probas):
    h_next, h1_next, next_logprobs = L.get_output([new_rnn,new_rnn1, next_token_logprobs],
                           {
                               #send x_t and h_t to the appropriate output
                               prev_token: x_t,
                               prev_rnn: h_t,
                               prev_rnn1: h1_t
                           })
    
    return h_next, h1_next, next_logprobs

(h_seq, h1_seq, predicted_logprobas), upd = theano.scan(rnn_one_step, 
                                        outputs_info=state0, sequences=input_sequence)

predictions_matrix = T.reshape(predicted_logprobas[:-1],[-1,len(tokens)])
answers_flat = T.reshape(input_sequence[1:],[-1])

loss = -(predictions_matrix * T.extra_ops.to_one_hot(answers_flat, n_tokens)).sum(axis=-1).mean()

weights =  L.get_all_params([new_rnn,next_token_probs])
all_grads = T.grad(loss, weights)
scaled_grads = lasagne.updates.total_norm_constraint(all_grads, 100)
optimizer = lasagne.updates.adam(scaled_grads, weights)


train_step = theano.function([input_sequence], loss, updates=upd + optimizer)

x_t = T.ivector('previous tokens')
h_t = theano.shared(np.zeros([1,rnn_num_units],'float32'))
h1_t = theano.shared(np.zeros([1,rnn_num_units],'float32'))

h_next,h1_next,next_logprobs = rnn_one_step(x_t,h_t,h1_t,probas0)
temp = theano.shared(np.float32(1))
next_probs=T.nnet.softmax(next_logprobs/temp)

update_rnn = theano.function([x_t], next_probs,
                           updates={h_t : h_next,
                                   h1_t:h1_next},
                               allow_input_downcast=True)

def generate_sample(seed_phrase=' ',max_length=MAX_LENGTH):
    '''
    The function generates text given a phrase of length at least SEQ_LENGTH.
        
    parameters:
        The phrase is set using the variable seed_phrase
        The optional input "N" is used to set the number of characters of text to predict.     
    '''
    if(seed_phrase!=' '):
        x_sequence = [token_to_id[token] for token in seed_phrase.split()]
    else:
        x_sequence = [token_to_id[" "]]
    
    h_t.set_value(np.zeros([1,rnn_num_units],'float32'))
    h1_t.set_value(np.zeros([1,rnn_num_units],'float32'))
    
    #feed the seed phrase, if any
    for ix in x_sequence[:-1]:
         _ = update_rnn([ix])
    
    #start generating
    for _ in range(max_length-len(seed_phrase)):
        x_probs = update_rnn([x_sequence[-1]])
        x_sequence.append(np.random.choice(n_tokens,p=x_probs[0]))
        
    return ' '.join([tokens[ix] for ix in x_sequence])

def save():
    np.savez("weights_LSTM-words.npz",*L.get_all_param_values([new_rnn,next_token_probs]))
    
def load():
    with np.load("weights_LSTM-words.npz") as f:
        param_values = [f['arr_%d' % i] for i in range(len(f.files))]
    L.set_all_param_values([new_rnn,next_token_probs],param_values)
    
load()

#temp.set_value(np.float32(0.5))

from IPython.display import clear_output
from random import sample
history = []
from tqdm import tqdm
temp.set_value(np.float32(1))
history=[]

def learn(a):
    for i in tqdm(range(a)):
        batch = to_matrix(sample(names,32),max_len=MAX_LENGTH)
        loss_i = train_step(batch)
    
    
        history.append(loss_i)
        if (i+1)%100==0:
            save()
            clear_output(True)
            plt.plot(history,label='loss')
            plt.legend()
            plt.show() 
            for _ in range(10):
                print(generate_sample())

 спорт: В Нефтеюганске чествовали юных акробатов
 Warhammer: Новое дополнение к Total War: Rome II вернёт игроков в античность
 политика: К ЧМ-2018 Ростводоканал внедрит систему глубокой очистки воздуха
 культура: Дмитров полностью подготовили к женскому чемпионату мира по хоккею
 наука: В Москве активно развивается предпрофессиональное образование
 Dota: Обзор LoL Catalyst. Помощник новичку в Лиге Легенд
 терроризм: Криминал гуляет у границ с Монголией и Китаем
 футбол: Новый понтонный мост установили на Мещерском озере
Seems alright!
    100% |################################| 235kB 2.1MB/s ta 0:00:01
  Running setup.py install for Lasagne ... done
     - 11.7MB 96.0MB/ss
  Running setup.py install for agentnet ... done
env: THEANO_FLAGS=floatX=float32


In [ ]:
import ipywidgets as widgets
from IPython.display import display
result = widgets.Textarea()
seed=widgets.Text()
temp_slider=widgets.FloatSlider(min=0.001,max=2)
btn = widgets.Button(description='Press me')

def display_result(x):
    if not seed.value.startswith(' '):
        result.value=generate_sample(' '+seed.value)
    else:
        result.value=generate_sample(seed.value)
def temp_change(x):
    global temp
    temp.set_value(temp_slider.value)

temp_slider.on_trait_change(temp_change)
btn.on_click(display_result)

In [ ]:
display(btn,seed,temp_slider,result)